In [49]:
import numpy as np
import pandas as pd
import math
import re
from collections import Counter
from functools import cmp_to_key
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache

In [6]:
with open("14-input", "r") as file:
    lines = file.readlines()
data_raw = [line.replace("\n", "") for line in lines]
data_raw = "\n".join(data_raw)
data_raw

'#....#O##.....O.#...##...#OO.....##...#O...O..........OO......#..OO..O.O...O.O#O#..O...#.......O...#\nO.....O#..O#O...OO.#.....O#.....O.O..O.O..O.O#.O.#...#...O.#.........#..#O.#......#O.O...#..#.##.##.\n......##..#.O#..#.O.O......#.O#..O....#.#..#...#.O.#....#....O.#...#O#OOO.##...O............OO#.#..#\n.OO.................#.#.O.#O.O.#...#..........##.O..#.O##..##O......#O...#.#...O...#......O.........\n#.......O#...#.OO.##.O.......#...O......O#......#..#.#..O...##..O#..#O.O.....#..OO..O.O.....O.....O#\n.O..O.OO...#...#O####...##.O.#..O......#O.O..O......O.......O....O....#..O...O.....OOO..#...O.....OO\nOO.#O.....#.O.#O.#OOO...OO#.........#..O.#O.#O..##O...#.##..#.....#...#.....O...O...............O.OO\n...OO..#....O.#O.....#.#.#..O...O.#....#....#O...O.........O..##O....O#.O.......OO#.O.O..O......OO..\n...O....#....#.....O...O.#.O.O...O..#..#.O.....O.#....O#O#.##O....O...#....#......#.....#..O....#..O\nOOOO#..##...##.....OO..O##.O.OOO#..O...O.#....O..OO##......O..#.O.#.....O.O#.#.#.

In [4]:
test_data_raw = """O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#...."""



def preprocess_data (data):
    return np.array([list(row)  for row in data.split("\n")], dtype='U10')

test_data = preprocess_data(test_data_raw)
test_data

array([['O', '.', '.', '.', '.', '#', '.', '.', '.', '.'],
       ['O', '.', 'O', 'O', '#', '.', '.', '.', '.', '#'],
       ['.', '.', '.', '.', '.', '#', '#', '.', '.', '.'],
       ['O', 'O', '.', '#', 'O', '.', '.', '.', '.', 'O'],
       ['.', 'O', '.', '.', '.', '.', '.', 'O', '#', '.'],
       ['O', '.', '#', '.', '.', 'O', '.', '#', '.', '#'],
       ['.', '.', 'O', '.', '.', '#', 'O', '.', '.', 'O'],
       ['.', '.', '.', '.', '.', '.', '.', 'O', '.', '.'],
       ['#', '.', '.', '.', '.', '#', '#', '#', '.', '.'],
       ['#', 'O', 'O', '.', '.', '#', '.', '.', '.', '.']], dtype='<U10')

In [7]:
data = preprocess_data(data_raw)
data

array([['#', '.', '.', ..., '.', '.', '#'],
       ['O', '.', '.', ..., '#', '#', '.'],
       ['.', '.', '.', ..., '.', '.', '#'],
       ...,
       ['O', '.', '.', ..., 'O', '#', 'O'],
       ['.', '.', '#', ..., '.', '#', '.'],
       ['.', '.', '.', ..., '.', '.', 'O']], dtype='<U10')

In [32]:


def solution (data, verbose=False):
    
    def tilt_up (data):
        data_tilted = np.zeros_like(data)
        for col_idx in range(data.shape[1]):
            parts = "".join(data[:, col_idx].tolist()).split("#")
            parts = ["O"*part.count("O") + "."*part.count(".") for part in parts]
            data_tilted[:, col_idx] = np.array([list("#".join(parts)) ], dtype='U10')
        return data_tilted

    def calc_loads (data):
        loads = []
        for row_idx in range(data.shape[0]):
            multiplier = data.shape[0] - row_idx
            loads.append(np.count_nonzero(data[row_idx, :] == "O") * multiplier)

        return loads


    data_tilted = tilt_up(data)
    loads = calc_loads(data_tilted)

    return sum(loads)


sol = solution(data)
# sol = solution(test_data, verbose=True)

print(sol)
# display(sum(sol))


108144


# Part 2

In [76]:
def decode_string_to_numpy (data):
    return np.array([list(row) for row in data.split("\n")], dtype='U10')

def encode_numpy_to_string(data):
    return "\n".join(["".join(row) for row in data.tolist()])

def solution (data, verbose=False):
    

    def tilt_up (data):
        data_tilted = np.zeros_like(data)
        for col_idx in range(data.shape[1]):
            parts = "".join(data[:, col_idx].tolist()).split("#")
            parts = ["O"*part.count("O") + "."*part.count(".") for part in parts]
            data_tilted[:, col_idx] = np.array([list("#".join(parts)) ], dtype='U10')
        return data_tilted

    def calc_loads (data):
        loads = []
        for row_idx in range(data.shape[0]):
            multiplier = data.shape[0] - row_idx
            loads.append(np.count_nonzero(data[row_idx, :] == "O") * multiplier)

        return loads

    @cache
    def one_cycle (data_encoded):
        data_decoded = decode_string_to_numpy(data_encoded)
        for cycle_step in range(4):
            data_decoded = tilt_up(data_decoded)
            data_decoded = np.rot90(data_decoded, 3)
        return data_decoded



    # num_cycles = 1000000000
    # num_cycles = 3
    num_cycles = 10000 * 10
    data_tilted = data.copy()
    data_save = []
    loads_save = []
    total_save = []

    total_load = -1

    for cycle in range(num_cycles):
        if verbose:
            if cycle % 10000 == 0:
                print(f"{cycle / num_cycles * 100:.2f}", "% done")
        
        data_encoded = encode_numpy_to_string(data_tilted)
        data_tilted = one_cycle(data_encoded)
        loads = calc_loads(data_tilted)
        total_load = sum(loads)
        found_match = False
        match_index = -1
        for index in [i for i, x in enumerate(total_save) if x == total_load]:
            if loads == loads_save[index] and np.array_equal(data_tilted, data_save[index]):
                found_match = True 
                match_index = index
                break

        if not found_match: 
            data_save.append (data_tilted)
            loads_save.append(loads)
            total_save.append(total_load)
        else:
            total_save_cycle = total_save[match_index:]
            index_in_cylce = ((1000000000-1) - match_index) % len(total_save_cycle)
            if verbose:
                print("Cycle from index:", match_index)
                print(total_save_cycle)
                print(data_save[match_index])
                print(data_tilted)
                print("Index in cycle", index_in_cylce)
                print("total load", total_save_cycle[index_in_cylce])

            total_load = total_save_cycle[index_in_cylce]
            break

        # print(loads, sum(loads))
        # print("Cylce", cycle, "\n", data_tilted)
    

    return total_load


sol = solution(data, verbose=True)
# sol = solution(test_data, verbose=True)

print(sol)
# display(sum(sol))


0.00 % done
Cycle from index: 114
[108472, 108455, 108440, 108424, 108410, 108404, 108425, 108440, 108472, 108481, 108483, 108467, 108460, 108435, 108429, 108405, 108409, 108420, 108445, 108467, 108486, 108478]
[['#' '.' '.' ... 'O' 'O' '#']
 ['.' '.' '.' ... '#' '#' '.']
 ['.' '.' '.' ... '.' '.' '#']
 ...
 ['.' '.' 'O' ... '.' '#' '.']
 ['O' 'O' '#' ... 'O' '#' '.']
 ['.' '.' '.' ... 'O' 'O' 'O']]
[['#' '.' '.' ... 'O' 'O' '#']
 ['.' '.' '.' ... '#' '#' '.']
 ['.' '.' '.' ... '.' '.' '#']
 ...
 ['.' '.' 'O' ... '.' '#' '.']
 ['O' 'O' '#' ... 'O' '#' '.']
 ['.' '.' '.' ... 'O' 'O' 'O']]
Index in cycle 5
total load 108404
108404
